# Import Modules

In [ ]:
import importlib
import h5py as h5
import pathlib

import numpy as np
from scipy import signal, stats, interpolate, ndimage

import matplotlib.pyplot as plt
import seaborn as sns

import userfunctions as uf
import fightclassifier as fac
import clusteranalysis as cn

# Import Datasets

In [ ]:
main_path = pathlib.Path().absolute()

In [ ]:
data_name = input('Name of dataset to be used: ')
datadir = main_path+data_name
data = h5.File(datadir,'r')
print(list(data.keys()))

In [ ]:
positions = data['tracks_3D_smooth']
positions_filt, N_NaN = uf.filt_trajectories(positions)

# Compute kinematic variables and behavioral states

## If datasets not available

In [ ]:
Single_v,Comp_v = fac.kinematic_analysis(positions_filt[:],0.01)

In [ ]:
State_dict = fac.fight_analysis(Single_v,Comp_v)

### Compute ethogram_array

In [ ]:
len_p = len(positions_filt)
n_states_in_dict = 6

State_array = np.zeros((n_states_in_dict,len_p))

In [ ]:
State_array[0] = State_dict['passive display']
State_array[1] = State_dict['circling']
State_array[2] = State_dict['aggressive state'][:,0]
State_array[3] = State_dict['aggressive state'][:,1]
State_array[4] = State_dict['freezing'][:,0]
State_array[5] = State_dict['freezing'][:,1]

In [ ]:
ethogram_array = fac.compute_etho_array(State_array,100)

## If datasets available

In [ ]:
single_v_name = input('Name of dataset with single fish variables: ') #with .h5
svdir = main_path + single_v_name
Single_v = h5.File(svdir,'r')

joint_v_name = input('Name of dataset with joint variables: ') #with .h5
jvdir = main_path + joint_v_name
Comp_v = h5.File(jvdir,'r')

state_v_name = input('Name of dataset with compound behavioral states: ') #with .h5
stdir = main_path + state_v_name
ethogram_dict = h5.File(stdir,'r')

# Save computed datasets

## Save single variables 

In [ ]:
dataset_name = input('Name of dataset to be saved: ')
dataset_name = dataset_name + '.h5'

In [ ]:
bevstate = h5.File(dataset_name,'w')
bevstate.create_dataset('distance',data=Single_v['distance'])
bevstate.create_dataset('speed',data=Single_v['speed'])
bevstate.create_dataset('heading',data=Single_v['heading'])
bevstate.create_dataset('z position',data=Single_v['z position'])
bevstate.create_dataset('tail beat rate',data=Single_v['tail beat rate'])
bevstate.create_dataset('tail z bend',data=Single_v['tail z bend'])
bevstate.close()

## Save joint variables

In [ ]:
dataset_name = input('Name of dataset to be saved: ')
dataset_name = dataset_name + '.h5'

In [ ]:
bevstate = h5.File(dataset_name,'w')
bevstate.create_dataset('average distance',data=Comp_v['average distance'])
bevstate.create_dataset('mean speed',data=Comp_v['mean speed'])
bevstate.create_dataset('alignment',data=Comp_v['alignment'])
bevstate.create_dataset('acc alignment',data=Comp_v['acc alignment'])
bevstate.create_dataset('mean tail beating rate',data=Comp_v['mean tail beating rate'])
bevstate.create_dataset('var tail beating rate',data=Comp_v['var tail beating rate'],axis=1))
bevstate.close()

## Save compound behav states

In [ ]:
dataset_name = input('Name of dataset to be saved: ')
dataset_name = dataset_name + '.h5'

In [ ]:
bevstate = h5.File(dataset_name,'w')
bevstate.create_dataset('Passive display',data=ethogram_array[0])
bevstate.create_dataset('Active display',data=ethogram_array[1])
bevstate.create_dataset('Sym Fight',data=ethogram_array[2])
bevstate.create_dataset('Asym Fight',data=ethogram_array[3])
bevstate.create_dataset('Freeze',data=ethogram_array[4])
bevstate.create_dataset('Other',data=ethogram_array[5])
bevstate.close()

# Analysis of compound behavioral states

## Variable averages in different compound behavioral states

In [ ]:
comp_v_array = np.zeros((6,len_p))
comp_v_array[0] = Comp_v['average distance']
comp_v_array[1] = Comp_v['mean speed']
comp_v_array[2] = np.arcos(Comp_v['alignment'])
comp_v_array[3] = np.arcos(Comp_v['acc alignment'])
comp_v_array[4] = Comp_v['mean tail beating rate']/(2*np.pi)
comp_v_array[5] = Comp_v['var tail beating rate']/(2*np.pi)**2

In [ ]:
comp_v_avg = cn.cluster_bev_avg(comp_v_array,ethogram_array.T)

## Behavioral states as a function of tail beat rate

In [ ]:
att_state,tb_bins = uf.bin_xy(comp_v_array[4]/(2*np.pi),np.sum(ethogram_array[2:4],axis=0),500)
freeze_state,tb_bins = uf.bin_xy(comp_v_array[4]/(2*np.pi),ethogram_array[4],500)
disp_state,tb_bins = uf.bin_xy(comp_v_array[4]/(2*np.pi),np.sum(ethogram_array[:2],axis=0),500)

In [ ]:
state_vec = np.zeros((400,3))
state_vec[:,0] = disp_state[:400]/np.max(disp_state[:400])
state_vec[:,1] = att_state[:400]/np.max(att_state[:400])
state_vec[:,2] = freeze_state[:400]/np.max(att_state[:400])

sum_st = np.sum(state_vec,axis=1)
state_vec1 = np.zeros(state_vec.shape)
state_vec1[:,0] = state_vec[:,0]/sum_st
state_vec1[:,1] = state_vec[:,1]/sum_st
state_vec1[:,2] = state_vec[:,2]/sum_st

### Fit generalized linear models to the data

In [ ]:
beta_ag, alpha_ag = uf.gen_linear_mod_fit(tb_bins[100:200],state_vec1[100:200,1])
beta_fr, alpha_fr = uf.gen_linear_mod_fit(tb_bins[30:100],state_vec1[30:100,2])

In [ ]:
f_ag= 1/(1+np.exp(-beta_ag*tb_bins+alpha_ag))
f_fr = 1/(1+np.exp(-beta_fr*tb_bins+alpha_fr))
f_dsp = 1 - f_ag - f_fr

w_ag = -alpha_ag/beta_ag
w_fr = -alpha_fr/beta_fr

## Behavioral states as a function of tail beat rate and distance

In [ ]:
Atk_r, dist_r,z_r = uf.bin_xyz(comp_v_array[0],comp_v_array[4],np.sum(ethogram_array[2:4],axis=0),50)
Frz_r, dist_r,z_r = uf.bin_xyz(comp_v_array[0],comp_v_array[4],ethogram_array[4],50)
Dsp_r, dist_r,z_r = uf.bin_xyz(comp_v_array[0],comp_v_array[4],np.sum(ethogram_array[:2],axis=0),50)

In [ ]:
H_dtb = np.histogram2d(comp_v_array[0],comp_v_array[4],bins=50)[0]
H_f = ndimage.gaussian_filter(H_dtb,1)

# Deviation of the tail beat rate and synchronization

In [ ]:
w = Single_v['tail beat rate']
phase = np.zeros(w.shape)
dt = 0.01
for i in range(2):
    phase[:,i] = np.cumsum(w[:,i])*dt
    
sig1 = np.exp(1j*phase)
sig_fut = sig1[1:]

In [ ]:
# Build a linear model
win_mod = 3000
B_vec, r_vec, Corr_v, Corr_est = uf.LinModv1(np.real(sig1[:-1]),np.real(sig_fut),win_mod)

In [ ]:
b_coef = B_vec[:,1:,0]/dt
A_coef = B_vec[:,1:,1:]
K_est = (1-np.linalg.det(A_coef))/(dt)

w_avg = comp_v_array[4]
w_var = comp_v_array[5]
w_dev_500 = w_var[:]**0.5/w_avg[:]

w_wind = []
for i in range(len(phase)//win_mod):
    w_wind.append(np.median(w_dev_500[i*win_mod:(i+1)*win_mod]))
w_wind = np.array(w_wind)



In [ ]:
smooth_w = uf.windowed_average(w_wind,3)
smooth_k = uf.windowed_average(K_est,3)

W_fit, K_fit = uf.bin_xy(smooth_k,smooth_w,25)

In [ ]:
ppwk = np.histogram2d(K_est,w_wind,50)
fwk = ndimage.gaussian_filter(ppwk[0],2)

# Visualization

## Display ethogram

In [ ]:
plt.figure(figsize=(16,4))
plt.pcolormesh(np.arange(6),np.arange(len_p),np.log(ethogram_array),cmap='Blues')
plt.gca().invert_yaxis()
plt.yticks(np.arange(6),['d1','d2','a1','a2','f','o'])
plt.title('Ethogram')

## Averages of variables in different states

In [ ]:
N = len(ethogram_array)

plt.figure(figsize=(16,8))
plt.subplot(231)
plt.bar(np.arange(1,N+1),avg_var[4],alpha=0.5,yerr=avg_var[4]/10,color='gold')
plt.title('Mean 2$\pi\omega(s^{-1})$',fontsize=20)
plt.xticks(np.arange(1,N+1),['d1','d2','a1','a2','o'])
plt.grid()

plt.subplot(232)
plt.bar(np.arange(1,N+1),avg_var[0],alpha=0.5,yerr=avg_var[0]/10,color='orangered')
plt.title('distance (bl)',fontsize=20)
plt.xticks(np.arange(1,N+1),['d1','d2','a1','a2','o'])
plt.grid()

plt.subplot(233)
plt.bar(np.arange(1,N+1),avg_var[3],alpha=0.5,yerr=avg_var[3]/10,color='cornflowerblue')
plt.title('speed (bl/s)',fontsize=20)
plt.xticks(np.arange(1,N+1),['d1','d2','a1','a2','o'])
plt.grid()

plt.subplot(234)
plt.bar(np.arange(1,N+1),avg_var[1],alpha=0.5,yerr=avg_var[1]/10,color='green')
plt.xticks(np.arange(1,N+1),['d1','d2','a1','a2','o'])
plt.title('alignment',fontsize=20)

plt.subplot(235)
plt.bar(np.arange(1,N+1),avg_var[2],alpha=0.5,yerr=avg_var[2]/10,color='limegreen')
plt.xticks(np.arange(1,N+1),['d1','d2','a1','a2','o'])
plt.title('Acc. alignment',fontsize=20)

plt.subplot(236)
plt.bar(np.arange(1,N+1), avg_var[5], alpha=0.5,yerr=avg_var[5]/10,color='goldenrod')
plt.title('Dev $\omega$',fontsize=20)
plt.xticks(np.arange(1,N+1),['d1','d2','a1','a2','o'])
plt.plot
plt.grid()

## Beh. states as functions of tail beat, with fits

In [ ]:
plt.figure(figsize=(16,8))

plt.plot(tb_bins,f_dsp,'--',color='green')
plt.plot(tb_bins,f_ag,'--',color='orangered')
plt.plot(tb_bins,f_fr,'--',color='blue')

plt.plot(tb_bins,frz_state[:])
plt.plot(tb_bins,agg_state[:])
plt.plot(tb_bins,disp_state[:])

plt.ylim(0,1)
plt.grid()

plt.legend(['Display','Aggressive state','Freezing'],fontsize=16)
plt.xlabel('$\\hat{\omega}$ $(s^{-1})$ ',fontsize=20)
plt.title('Relative frequency (fitted)', fontsize=22)

## Deviation of tail beat rate in function of the coupling between the pair

In [ ]:
plt.contourf(ppwk[1][:-1],ppwk[2][:-1],fwk.T,cmap='jet')
plt.plot(K_fit,W_fit,'k')
plt.xlabel('Coupling constant',fontsize=16)
plt.ylabel('Tail beat Deviation',fontsize=16)

## Beh. States as function of tail beat and distance

In [ ]:
plt.figure(figsize=(20,5))

D = ndimage.gaussian_filter(Dsp_r,1)
A = ndimage.gaussian_filter(Atk_r,1)
F = ndimage.gaussian_filter(Frz_r,1)

plt.subplot(131)
plt.title('Display',fontsize=16)
plt.contourf(dist_r,z_r,D/np.sum(D),cmap='jet')
plt.xlabel('Distance (bl)')
plt.ylabel('Mean $\omega$ ($s^{-1}$)')
plt.colorbar()

plt.subplot(132)
plt.title('Aggressive state',fontsize=16)
plt.contourf(dist_r,z_r,A/np.sum(A),cmap='jet')
plt.colorbar()
plt.xlabel('Distance (bl)')
plt.ylabel('Mean $\omega$ ($s^{-1}$)')

plt.subplot(133)
plt.title('Freezing',fontsize=16)
plt.contourf(dist_r,z_r,F/np.sum(F),cmap='jet')
plt.colorbar()
plt.xlabel('Distance (bl)')
plt.ylabel('Mean $\omega$ ($s^{-1}$)')
plt.show()

In [ ]:
plt.contourf(dist_r,z_r/(2*np.pi),(H_f.T)/np.sum(H_f),15,cmap='jet')
plt.xlabel('Distance (bl)')
plt.ylabel('Mean $\omega$ ($s^{-1}$)')
plt.title('Histogram',fontsize=16)
plt.colorbar()